In [11]:
import time
import threading
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

In [12]:
edge_options = Options()
edge_options.add_argument('--headless')
edge_options.add_argument('--disable-gpu')
edge_options.add_argument('--no-sandbox')
edge_options.add_argument('start-maximized')
service = Service()

In [13]:
num_queries = int(input("How many product searches do you want to perform? "))

In [14]:
search_queries = []
for i in range(num_queries):
    query = input(f"Enter search query {i+1}: ").strip().replace(' ', '+')  # Replacing spaces with '+'
    search_queries.append(f'https://www.amazon.com/s?k={query}')

In [15]:
threads = []
results = []

for url in search_queries:
    def scrape_task(url=url):
        # Initialize WebDriver for each thread
        driver = webdriver.Edge(service=service, options=edge_options)
        driver.get(url)
        time.sleep(2)  # Wait for the page to load completely
        page_source = driver.page_source
        driver.quit()

        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Extract product information
        product_list = []
        for item in soup.select('div.s-main-slot div[data-asin]'):
            name = item.select_one('span.a-size-medium').get_text(strip=True) if item.select_one('span.a-size-medium') else None
            price = item.select_one('span.a-price-whole').get_text(strip=True) if item.select_one('span.a-price-whole') else None
            rating = item.select_one('span.a-icon-alt').get_text(strip=True) if item.select_one('span.a-icon-alt') else None
            reviews = item.select_one('span.a-size-base').get_text(strip=True) if item.select_one('span.a-size-base') else None
            product_list.append([name, price, rating, reviews])

        # Append product data to results
        results.extend(product_list)

    thread = threading.Thread(target=scrape_task)
    threads.append(thread)
    thread.start()

In [16]:
for thread in threads:
    thread.join()

In [17]:
with open('amazon_products.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Price', 'Rating', 'Reviews'])  # Header
    writer.writerows(results)

print('Saved data to amazon_products.csv')

Saved data to amazon_products.csv


In [1]:
import os
os.getcwd()

'c:\\Users\\Mohan'